In [ ]:
# =============================
#   0. 安装依赖
# =============================
!pip install --quiet transformers rouge-score sentencepiece


In [ ]:
# =============================
#   1. 导入库
# =============================
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch.optim import AdamW
from tqdm import tqdm
from rouge_score import rouge_scorer
import os, random

In [ ]:
# =============================
#   2. 读取数据
# =============================
train_path = "/kaggle/input/nanogpt-fudannlp-cs-30040/train.csv"
test_path  = "/kaggle/input/nanogpt-fudannlp-cs-30040/test.csv"
save_dir = "/kaggle/working/out-summarization/t5"
os.makedirs(save_dir, exist_ok=True)


df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [ ]:
# =============================
#   3. 划分训练 / 验证集 (90/10)
# =============================
val_ratio = 0.1
val_size = int(len(df) * val_ratio)
train_size = len(df) - val_size

train_df, val_df = random_split(df, [train_size, val_size])
train_df = df.iloc[train_df.indices]
val_df = df.iloc[val_df.indices]

len(train_df), len(val_df)

In [ ]:
# =============================
#   4. 定义 Dataset
# =============================
tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-small")


class SummarizationDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        inp = "summarize: " + row["dialogue"]
        tgt = row["summary"]

        enc = tokenizer(inp, truncation=True, padding="max_length", max_length=512, return_tensors="pt")
        dec = tokenizer(tgt, truncation=True, padding="max_length", max_length=128, return_tensors="pt")

        return {
            "input_ids": enc["input_ids"].squeeze(),
            "attention_mask": enc["attention_mask"].squeeze(),
            "labels": dec["input_ids"].squeeze(),
        }


train_ds = SummarizationDataset(train_df)
val_ds = SummarizationDataset(val_df)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32)

In [ ]:
# =============================
#   5. 加载模型
# =============================
from transformers import T5ForConditionalGeneration, T5Tokenizer
from torch.optim import AdamW

device = "cuda" if torch.cuda.is_available() else "cpu"

def load_t5_model(load_local=False, local_path=None):
    if load_local and local_path is not None:
        print(f"🔄 从本地加载模型: {local_path}")
        model = T5ForConditionalGeneration.from_pretrained(local_path)
    else:
        print("🌐 从 Hugging Face 加载: google-t5/t5-small")
        model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small")
    return model.to(device)


load_local = True # 是否从本地加载模型
local_model_dir = "/kaggle/working/out-summarization/t5"

model = load_t5_model(load_local, local_model_dir)
tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-small")
optimizer = AdamW(model.parameters(), lr=1e-5)

In [ ]:
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)


def print_sample_rouge():
    sample = val_df.sample(10)
    r1 = r2 = rL = 0

    for _, row in sample.iterrows():
        text = "summarize: " + row["dialogue"]
        ref = row["summary"]

        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
        pred_ids = model.generate(**inputs, max_length=128)
        pred = tokenizer.decode(pred_ids[0], skip_special_tokens=True)

        s = scorer.score(ref, pred)
        r1 += s["rouge1"].fmeasure
        r2 += s["rouge2"].fmeasure
        rL += s["rougeL"].fmeasure

    print(f"[ROUGE] R1={r1/10:.4f}  R2={r2/10:.4f}  RL={rL/10:.4f}  Score={(r1+r2+rL)/30:.4f}")

In [ ]:
# =============================
#   6. 训练（简洁版梯度累计）
# =============================

best_val_loss = 0.3810
EPOCHS = 10
log_interval = 10
accumulation_steps = 8  # 梯度累计步数
step = 0
global_step = 0  # 实际参数更新步数

for epoch in range(EPOCHS):
    print(f"===== Epoch {epoch+1} =====")
    model.train()

    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}")

    optimizer.zero_grad()  # 在epoch开始时清零梯度

    for i, batch in enumerate(pbar):
        batch = {k: v.to(device) for k, v in batch.items()}
        out = model(**batch)
        loss = out.loss

        # 归一化损失并反向传播
        loss = loss / accumulation_steps
        loss.backward()

        # 累计步数达到时更新参数
        if (i + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1

            # 显示信息
            pbar.set_postfix({"loss": f"{loss.item() * accumulation_steps:.4f}", "global_step": global_step})

            # 验证逻辑
            if global_step % log_interval == 0:
                model.eval()
                val_loss = 0

                with torch.no_grad():
                    for vb in val_loader:
                        vb = {k: v.to(device) for k, v in vb.items()}
                        out = model(**vb)
                        val_loss += out.loss.item()

                val_loss /= len(val_loader)

                pbar.set_postfix({"loss": f"{loss.item() * accumulation_steps:.4f}", "val_loss": f"{val_loss:.4f}", "global_step": global_step})

                print_sample_rouge()

                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    model.save_pretrained(save_dir)
                    print(f"✨ Saved best model (val_loss = {best_val_loss:.4f})\n")

                model.train()

    pbar.close()

In [ ]:
# =============================
#   7. 测试集推理
# =============================
model.eval()

summaries = []

for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
    text = "summarize: " + row["dialogue"]
    inp = tokenizer(text, return_tensors="pt", truncation=True).to(device)

    pred_ids = model.generate(**inp, max_length=128)
    pred = tokenizer.decode(pred_ids[0], skip_special_tokens=True)

    summaries.append(pred)

out = pd.DataFrame(
    {
        "id": test_df["id"],
        "summary": summaries,
    }
)

out_path = f"{save_dir}/submission.csv"
out.to_csv(out_path, index=False)